![Parked motorcycle](motorcycle.jpg)

You're working for a company that sells motorcycle parts, and they've asked for some help in analyzing their sales data!

They operate three warehouses in the area, selling both retail and wholesale. They offer a variety of parts and accept credit cards, cash, and bank transfer as payment methods. However, each payment type incurs a different fee.

The board of directors wants to gain a better understanding of wholesale revenue by product line, and how this varies month-to-month and across warehouses. You have been tasked with calculating net revenue for each product line and grouping results by month and warehouse. The results should be filtered so that only `"Wholesale"` orders are included.

Using the warehouse data provide sale targets for each warehouse and create a dashboard with this data. 

They have provided you with access to their database, which contains the following table called `sales`:

## Sales
| Column | Data type | Description |
|--------|-----------|-------------|
| `order_number` | `VARCHAR` | Unique order number. |
| `date` | `DATE` | Date of the order, from June to August 2021. |
| `warehouse` | `VARCHAR` | The warehouse that the order was made from&mdash; `North`, `Central`, or `West`. |
| `client_type` | `VARCHAR` | Whether the order was `Retail` or `Wholesale`. |
| `product_line` | `VARCHAR` | Type of product ordered. |
| `quantity` | `INT` | Number of products ordered. | 
| `unit_price` | `FLOAT` | Price per product (dollars). |
| `total` | `FLOAT` | Total price of the order (dollars). |
| `payment` | `VARCHAR` | Payment method&mdash;`Credit card`, `Transfer`, or `Cash`. |
| `payment_fee` | `FLOAT` | Percentage of `total` charged as a result of the `payment` method. |




In [24]:
-- Start coding here
SELECT product_line, 
	CASE WHEN EXTRACT('month' FROM date) = 6 -- Alternative date funcion TO_CHAR(date, 'Month') AS Month
			Then 'June'
		WHEN EXTRACT('month' FROM date) = 7 
			THEN 'July'
		WHEN EXTRACT('month' FROM date) = 8 
			THEN 'August'
		ELSE 'error'
	END AS month, 
	warehouse,  
	ROUND(SUM(cast(total* (1-payment_fee) AS numeric)) , 2) AS net_revenue
FROM public.sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, month, warehouse, warehouse
ORDER BY product_line, Month asc, net_revenue desc;

,product_line,month,warehouse,net_revenue
0,Breaking system,August,Central,3009.10
1,Breaking system,August,West,2475.71
2,Breaking system,August,North,1753.19
3,Breaking system,July,Central,3740.94
4,Breaking system,July,West,3030.39
5,Breaking system,July,North,2568.55
6,Breaking system,June,Central,3648.14
7,Breaking system,June,North,1472.93
8,Breaking system,June,West,1200.64
9,Electrical system,August,North,4673.99


# Improving Sales 

    In order to improve sales, we must identify our "top" selling and "lowest" selling items. Our plan of attack will be to target these two items inorder to increase average sales.
    
###     Sales Target:
-     Top selling items : Increase by 5% 
-     Lowest selling items : Increase by 10%
-     Total Average : Increase by 8% 

## Finding "Top" selling items per month and warehouse

    For this query, I identify the product that sells the best for each month and warehouse.  In order to do this, I created two subqueries. The first, brings the data from the initial project and the second subquery ranks items by their location and month. The final result is filtered in order to show only the "top" selling items. Target revenue column was added to show 5% sales target.
    The results will be used in our Tableau dashboard, so warehouse management can easily share pertenent sales data with their teams.

In [16]:
SELECT product_line,month, warehouse, new_revenue AS revenue_2021, Round((new_revenue * 1.05),0) AS Target_Revenue_2022
FROM
(
SELECT product_line, month, warehouse, ROUND(net_revenue,0) AS new_revenue, RANK() OVER( 
	PARTITION BY Month, warehouse
	ORDER BY net_revenue desc) AS Row_N  

FROM(SELECT product_line, 
	CASE WHEN EXTRACT('month' FROM date) = 6 -- Alternative date funcion TO_CHAR(date, 'Month') AS Month
			Then 'June'
		WHEN EXTRACT('month' FROM date) = 7 
			THEN 'July'
		WHEN EXTRACT('month' FROM date) = 8 
			THEN 'August'
		ELSE 'error'
	END AS month, 
	warehouse,  
	ROUND(SUM(cast(total* (1-payment_fee) AS numeric)) , 2) AS net_revenue
FROM public.sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, month, warehouse, warehouse
ORDER BY product_line, Month asc, net_revenue desc) AS OG_Table --Table from first SQL 
 ) AS top_sales

 WHERE row_n = '1'
 ORDER BY Month, warehouse;

,product_line,month,warehouse,revenue_2021,target_revenue_2022
0,Engine,August,Central,9433,9905
1,Frame & body,August,North,7820,8211
2,Breaking system,August,West,2476,2600
3,Suspension & traction,July,Central,6392,6712
4,Frame & body,July,North,6093,6398
5,Breaking system,July,West,3030,3182
6,Engine,June,Central,6483,6807
7,Suspension & traction,June,North,7985,8384
8,Frame & body,June,West,2752,2890


## Finding "Lowest" selling items per month and warehouse

    For this query, I identify the product that sells the best for each month and warehouse.  In order to do this, I created two subqueries. The first, brings the data from the initial project and the second subquery ranks items by their location and month. The final result is filtered in order to show only the "top" selling items. Target revenue column was added to show 5% sales target.
    The results will be used in our Tableau dashboard, so warehouse management can easily share pertenent sales data with their teams.

In [17]:
SELECT product_line,month, warehouse, new_revenue AS revenue_2021, Round((new_revenue * 1.10),0) AS Target_Revenue_2022
FROM
(
SELECT product_line, month, warehouse, ROUND(net_revenue,2) AS new_revenue, RANK() OVER( 
	PARTITION BY Month, warehouse
	ORDER BY net_revenue ) AS Row_N  

FROM(SELECT product_line, 
	CASE WHEN EXTRACT('month' FROM date) = 6 -- Alternative date funcion TO_CHAR(date, 'Month') AS Month
			Then 'June'
		WHEN EXTRACT('month' FROM date) = 7 
			THEN 'July'
		WHEN EXTRACT('month' FROM date) = 8 
			THEN 'August'
		ELSE 'error'
	END AS month, 
	warehouse,  
	ROUND(SUM(cast(total* (1-payment_fee) AS numeric)) , 2) AS net_revenue
FROM public.sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, month, warehouse, warehouse
ORDER BY product_line, Month asc, net_revenue desc) AS OG_Table --Table from first SQL 
 ) AS Lowest_sales

 WHERE row_n = '1'
 ORDER BY Month, warehouse;

,product_line,month,warehouse,revenue_2021,target_revenue_2022
0,Miscellaneous,August,Central,1722.40,1895
1,Breaking system,August,North,1753.19,1929
2,Miscellaneous,August,West,805.31,886
3,Engine,July,Central,1808.77,1990
4,Engine,July,North,997.08,1097
5,Electrical system,July,West,444.98,489
6,Miscellaneous,June,Central,1859.34,2045
7,Miscellaneous,June,North,508.86,560
8,Breaking system,June,West,1200.64,1321


## Finding Average Warehouse Sales By Month

    This query shows sales targets from a warehouse perspective by aggragating the average warehouse monthly sales.

In [18]:
SELECT month, warehouse, AVG_Revenue AS revenue_2021, ROUND(AVG_Revenue * 1.08,0) AS Target_AVG_Revenue_2022
FROM(
SELECT month, warehouse, ROUND(AVG(net_revenue),0) AS AVG_Revenue
FROM (
SELECT product_line, 
	CASE WHEN EXTRACT('month' FROM date) = 6 -- Alternative date funcion TO_CHAR(date, 'Month') AS Month
			Then 'June'
		WHEN EXTRACT('month' FROM date) = 7 
			THEN 'July'
		WHEN EXTRACT('month' FROM date) = 8 
			THEN 'August'
		ELSE 'error'
	END AS month, 
	warehouse,  
	ROUND(SUM(cast(total* (1-payment_fee) AS numeric)) , 2) AS net_revenue
FROM public.sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, month, warehouse, warehouse
ORDER BY product_line, Month asc, net_revenue desc) AS OG_TABLE

GROUP BY month, warehouse, warehouse
ORDER BY Month asc, AVG_Revenue desc) AS AVG_Revenue;


,month,warehouse,revenue_2021,target_avg_revenue_2022
0,August,Central,5199,5615
1,August,North,3874,4184
2,August,West,1280,1382
3,July,Central,3943,4258
4,July,North,2902,3134
5,July,West,1883,2034
6,June,Central,3870,4180
7,June,North,3366,3635
8,June,West,2140,2311


## Company Target

    This query shows sales targets from a company perspective by aggragating the average monthly sales for all warehouses.

In [25]:
SELECT Month, ROUND(AVG(net_revenue),0) AS AVG_Revenue_2021, ROUND((AVG(net_revenue) * 1.08),0) AS Target_AVG_Revenue_2022
FROM(
SELECT product_line, 
	CASE WHEN EXTRACT('month' FROM date) = 6 -- Alternative date funcion TO_CHAR(date, 'Month') AS Month
			Then 'June'
		WHEN EXTRACT('month' FROM date) = 7 
			THEN 'July'
		WHEN EXTRACT('month' FROM date) = 8 
			THEN 'August'
		ELSE 'error'
	END AS month, 
	warehouse,  
	ROUND(SUM(cast(total* (1-payment_fee) AS numeric)) , 2) AS net_revenue
FROM public.sales
WHERE client_type = 'Wholesale'
GROUP BY product_line, month, warehouse, warehouse
ORDER BY product_line, Month asc, net_revenue desc) AS OG_Table
Group BY month;

,month,avg_revenue_2021,target_avg_revenue_2022
0,June,3241,3500
1,August,3579,3865
2,July,3037,3280
